<a href="https://colab.research.google.com/github/MHmi1/smart-text-analysis-dj/blob/master/sentiment_analysis_deep_model(EN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW
from datasets import load_dataset

# Load the BART tokenizer and model
model_name = 'facebook/bart-base'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# Load the CNN/Daily Mail dataset (this may take a while if you're downloading it for the first time)
dataset = load_dataset('cnn_dailymail', '3.0.0', split='train')

# Get the source texts and summaries from the dataset
source_texts = dataset['article'][:100]   # Using the first 1000 samples for demonstration
summaries = dataset['highlights'][:100]   # Corresponding highlights are the abstractive summaries

# Tokenize the source texts and summaries
tokenized_inputs = tokenizer(source_texts, padding=True, truncation=True, return_tensors='pt')
tokenized_targets = tokenizer(summaries, padding=True, truncation=True, return_tensors='pt')

# Ensure inputs and targets have the same length
assert len(tokenized_inputs['input_ids']) == len(tokenized_targets['input_ids'])

# Prepare the DataLoader for faster training
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(tokenized_inputs['input_ids'], tokenized_inputs['attention_mask'],
                              tokenized_targets['input_ids'], tokenized_targets['attention_mask'])
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

# Fine-tuning loop with transfer learning
optimizer = AdamW(model.parameters(), lr=1e-7)
num_epochs = 2

model.train()

for epoch in range(num_epochs):
    total_loss = 0

    for batch in train_loader:
        batch_inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'decoder_input_ids': batch[2],
            'decoder_attention_mask': batch[3]
        }

        optimizer.zero_grad()

        outputs = model(**batch_inputs, labels=batch_inputs['decoder_input_ids'])
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()

    average_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs} - Average Loss: {average_loss:.4f}")

# Save the fine-tuned model
output_model_dir = './fine_tuned_model/'
model.save_pretrained(output_model_dir)
tokenizer.save_pretrained(output_model_dir)

# Rest of the code for summarization and website usage (unchanged)
# ...


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/2 - Average Loss: 11.0308
Epoch 2/2 - Average Loss: 10.7514


('./fine_tuned_model/tokenizer_config.json',
 './fine_tuned_model/special_tokens_map.json',
 './fine_tuned_model/vocab.json',
 './fine_tuned_model/merges.txt',
 './fine_tuned_model/added_tokens.json')

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

model_save_name = 'model.h5'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

Mounted at /content/gdrive
